<a href="https://colab.research.google.com/github/288756/VisArtificial/blob/master/Miniproyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive

# Montar el Google Drive en el directorio del proyecto y descomprimir el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/vision-artificial.zip' >> /dev/null  # ACTUALIZAR: ruta al fichero comprimido


Mounted at /content/gdrive


In [2]:
from sklearn.preprocessing import LabelEncoder

# Especificar las rutas al directorio con las imágenes y al fichero con las etiquetas
data_path = '/content/'
imgtrain_dir = data_path + "train/images/"
csvtrain_file = data_path + "train.csv"

imgtest_dir = data_path + "test/images/"
csvtest_file = data_path + "test.csv"
# Leer el fichero CSV con las etiquetas

dftrain = pd.read_csv(csvtrain_file, dtype = {"class": "category"})
dftest = pd.read_csv(csvtest_file, dtype = {"class": "category"})
# Codificar las etiquetas utilizando LabelEncoder
label_encoder = LabelEncoder()
dftrain['class_encoded'] = label_encoder.fit_transform(dftrain['class'])

# Convertir las etiquetas codificadas en un vector one-hot
class_one_hot = pd.get_dummies(dftrain['class_encoded'], prefix='class')

# Renombrar las columnas del vector one-hot
class_one_hot.columns = ['normal', 'benign', 'malignant']

# Concatenar el DataFrame original con las etiquetas one-hot
dftrain = pd.concat([dftrain[['image_filename']], class_one_hot], axis=1)

# Imprimir las primeras filas del DataFrame para verificar
print(len(imgtrain_dir))

22


In [3]:
val_size = int(len(dftrain) * 0.2)
dftrain = dftrain.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
dfval = dftrain[:val_size]
print(f'Número de ejemplos del conjunto de entrenamiento: {dftrain.shape[0]}')
print(f'Número de ejemplos del conjunto de validación: {dfval.shape[0]}')
print(f'Número de ejemplos del conjunto de test: {dftest.shape[0]}')
dftrain = dftrain.reset_index(drop=True)
dfval = dfval.reset_index(drop=True)

Número de ejemplos del conjunto de entrenamiento: 337
Número de ejemplos del conjunto de validación: 67
Número de ejemplos del conjunto de test: 113


In [4]:
import os
from PIL import Image

archivos = os.listdir(imgtrain_dir)
anchuras = []
alturas = []
for archivo in archivos:
      imagen = Image.open(os.path.join(imgtrain_dir, archivo))
      ancho, alto = imagen.size
      anchuras.append(ancho)
      alturas.append(alto)
media_anchura = sum(anchuras) / len(anchuras)
media_altura = sum(alturas) / len(alturas)

print("Media de anchura:", media_anchura)
print("Media de altura:", media_altura)

Media de anchura: 634.9139465875371
Media de altura: 514.9317507418398


In [11]:
import tensorflow as tf
from tensorflow.data import Dataset

# Dimensiones deseadas de la imagen
img_width, img_height = 635, 514
n_channels = 3                # número de canales (RGB)
n_classes = 3                 # número de clases
x_col = 'image_filename'      # nombres de las columnas en el fichero CSV
y_col = ['normal', 'benign', 'malignant']  # lista de nombres de las columnas de las etiquetas

# Cargar y preprocesar imágenes
def load_and_preprocess_image(image_filename, label_one_hot):
    image_path = tf.strings.join([imgtrain_dir, image_filename])
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=n_channels)
    # Redimensionar la imagen al tamaño deseado con relleno de ceros si es necesario
    image = tf.image.resize_with_pad(image, img_width, img_height)
    image = image / 255.0                               # normalización
    label = label_one_hot                                # Utilizar etiquetas codificadas como vector one-hot
    return image, label

# Crear conjunto de datos
def get_dataset(df):
    image_filenames = df[x_col].values
    labels = df[y_col].values
    dataset = tf.data.Dataset.from_tensor_slices((image_filenames, labels))
    dataset = dataset.map(load_and_preprocess_image)
    return dataset

# Crear los conjuntos de datos y preparar los lotes
batch_size = 256
train_dataset = get_dataset(dftrain).batch(batch_size)
val_dataset = get_dataset(dfval).batch(batch_size)

print(f'Número de lotes del conjunto de entrenamiento: {len(train_dataset)}')
print(f'Número de lotes del conjunto de validación: {len(val_dataset)}')


Número de lotes del conjunto de entrenamiento: 2
Número de lotes del conjunto de validación: 1


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models

def simple_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Define the input shape (width, height, channels)
input_shape = (img_width, img_height, 3)
# Define the number of classes
num_classes = 3  # Assuming you have 3 classes

# Create the model
model = simple_cnn_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use this loss function for integer labels
              metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 633, 512, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 316, 256, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 314, 254, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 157, 127, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 155, 125, 64)      36928     
                                                                 
 flatten_2 (Flatten)         (None, 1240000)          

In [13]:
import numpy as np

# Entrenar el modelo con los datos preparados previamente
history = model.fit(train_dataset,
          epochs=6,   # número de epochs
          verbose=2,  # muestra información al finalizar cada epoch
          validation_data=val_dataset)

# Imprimir el error mínimo de entrenamiento y validación
train_trace = np.array(history.history['loss'])
print(f'\nError mínimo en entrenamiento: {min(train_trace):.6f}')

val_trace = np.array(history.history['val_loss'])
print(f'Error mínimo en validación: {min(val_trace):.6f}')

Epoch 1/6


ResourceExhaustedError: Graph execution error:

Detected at node sequential_2/conv2d_6/Relu defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/

In [ ]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input

# Cargar el modelo previamente entrenado
model = get_model()  # Suponiendo que ya has definido y entrenado el modelo

# Directorio donde se encuentran las imágenes de prueba
test_images_dir = 'test/images/'

# Obtener la lista de nombres de archivos de las imágenes de prueba
test_filenames = os.listdir(test_images_dir)

# Crear un DataFrame para almacenar las predicciones
predictions_df = pd.DataFrame({'image_filename': test_filenames})

# Crear columnas para 'normal', 'benign' y 'malignant' con valores iniciales de 0
predictions_df['normal'] = 0
predictions_df['benign'] = 0
predictions_df['malignant'] = 0

# Iterar sobre cada imagen de prueba
for filename in test_filenames:
    # Cargar la imagen y preprocesarla
    img_path = os.path.join(test_images_dir, filename)
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # Añadir una dimensión adicional para el lote

    # Realizar la predicción
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)  # Obtener la clase predicha

    # Actualizar las columnas correspondientes según la predicción
    if predicted_class == 0:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'normal'] = 1
    elif predicted_class == 1:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'benign'] = 1
    elif predicted_class == 2:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'malignant'] = 1

# Ordenar el DataFrame por el nombre del archivo de imagen
predictions_df = predictions_df.sort_values(by='image_filename')

# Guardar el DataFrame en un archivo CSV
predictions_df.to_csv('test_predictions.csv', index=False)

# Imprimir las primeras filas del DataFrame para verificar
print(predictions_df.head())


Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0  